In [1]:
# Standard libraries
from time import strptime
import datetime

# Side libraries
import numpy as np
import holidays
import ipywidgets as widgets

# Initializing country list that contains all countries supported by 'holidays'
COUNTRIES = ['Argentina', 'Australia', 'Austria', 'Belarus',
             'Belgium', 'Brazil', 'Canada', 'Colombia', 'Croatia', 'Czech',
             'Denmark', 'England', 'Finland', 'France', 'Germany', 'Hungary',
             'India', 'Ireland', 'Isle of Man', 'Italy', 'Japan', 'Mexico',
             'Netherlands', 'NewZealand', 'Northern Ireland', 'Norway',
             'Polish', 'Portugal', 'Portugal Ext', 'Scotland', 'Slovenia',
             'Slovakia', 'South Africa', 'Spain', 'Sweden', 'Switzerland',
             'Ukraine', 'United Kingdom', 'United States', 'Wales']

PROVINCES = {
    'Australia':['ACT', 'NSW', 'NT', 'QLD', 'SA', 'TAS', 'VIC', 'WA'],
    'Austria':['B', 'K', 'N', 'O', 'S', 'ST', 'T', 'V', 'W'],
    'Brazil':['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT',
              'MS', 'MG', 'PA', 'PB', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR',
              'SC', 'SP', 'SE', 'TO'],
    'Canada':['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'NU', 'ON', 'PE', 'QC',
              'SK', 'YU'],
    'France':['Métropole', 'Alsace-Moselle', 'Guadeloupe', 'Guyane',
              'Martinique', 'Mayotte', 'Nouvelle-Calédonie', 'La Réunion',
              'Polynésie Française', 'Saint-Barthélémy', 'Saint-Martin',
              'Wallis-et-Futuna'],
    'Germany':['BW', 'BY', 'BE', 'BB', 'HB', 'HH', 'HE', 'MV', 'NI', 'NW', 'RP',
               'SL', 'SN', 'ST', 'SH', 'TH'],
    'India':['AS', 'SK', 'CG', 'KA', 'GJ', 'BR', 'RJ', 'OD', 'TN', 'AP', 'WB',
             'KL', 'HR', 'MH', 'MP', 'UP', 'UK', 'TN'],
    'Italy':['MI', 'RM'],
    'NewZealand':['NTL', 'AUK', 'TKI', 'HKB', 'WGN', 'MBH', 'NSN', 'CAN', 'STC',
                  'WTL', 'OTA', 'STL', 'CIT'],
    'Spain':['AND', 'ARG', 'AST', 'CAN', 'CAM', 'CAL', 'CAT', 'CVA', 'EXT',
             'GAL', 'IBA', 'ICA', 'MAD', 'MUR', 'NAV', 'PVA', 'RIO'],
    'Switzerland':['AG', 'AR', 'AI', 'BL', 'BS', 'BE', 'FR', 'GE', 'GL', 'GR',
                   'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SZ', 'SO', 'TG',
                   'TI', 'UR', 'VD', 'VS', 'ZG', 'ZH'],
    'UnitedStates':['AL', 'AK', 'AS', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC',
                    'FL', 'GA', 'GU', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY',
                    'LA', 'ME', 'MD', 'MH', 'MA', 'MI', 'FM', 'MN', 'MS', 'MO',
                    'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'MP',
                    'OH', 'OK', 'OR', 'PW', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
                    'TX', 'UT', 'VT', 'VA', 'VI', 'WA', 'WV', 'WI', 'WY']
}

def calculate_project_duration(fe_devs, fe_task, be_devs, be_task):
    '''
    Returns the number of days required to complete all tasks
    '''
    fe_hours = 0
    be_hours = 0
    # Calculating required number of hours for both front-end and back-end tasks
    try:
        if fe_task > 0:
            fe_hours = fe_task / fe_devs
        if be_task > 0:
            be_hours = be_task / be_devs
    except ZeroDivisionError:
        print('It seems like you have no enough developers to finish the task.')
        return -1

    # Calculating the number of days (rounding up without using a 'math')
    max_days = max(fe_hours, be_hours) / 8 + (max(fe_hours, be_hours) % 8 > 0)

    return int(max_days)

def get_holidays(country, state_prov, years):
    '''
    Creates a dictionary that contains all public national holidays for
    specific country, state or province and for period specified in 'years' list
    '''
    state = None
    province = None

    # Defining if there is any state or province specified
    if state_prov != '':
        if country in ['US', 'BR']:
            state = state_prov
        else:
            province = state_prov

    # Sending a request to 'holidays' library API
    holydays_list = getattr(holidays, country)(
        state=state,
        prov=province,
        years=years
    )

    return sorted([np.datetime64(hol.strftime('%Y-%m-%d')) for hol in holydays_list.keys()])

def main(fe_devs=1, fe_task=10, be_devs=1, be_task=10, start_date=datetime.datetime.now(), country='UnitedStates', state_prov='-NONE-', observe=True):
    '''
    The main function that calls all other ones and collect all the data
    '''

    # Calculating the project duration in days
    proj_days = calculate_project_duration(fe_devs, fe_task, be_devs, be_task)

    if proj_days == -1:
        print('Please try again.')
        return

    # Parsing the starting year from the project start date
    try:
        min_year = int(start_date.year)
    except:
        print('Error! Please enter a valid date.')
        return
    # Calculating the max project year based on project duration
    max_year = int(min_year + (proj_days / 260) + 1)

    # Getting the list (dict) of public holidays for specific region and years
    holidays_schedule = get_holidays(country.replace(' ', ''), state_prov, [y for y in range(min_year, max_year)])

    # Making an offset (is active by default)
    if observe:
        holidays_schedule = list(np.busday_offset(holidays_schedule, 0, roll='forward'))
    
    # Getting the end date of the project along with non-working days stats
    end_date = np.busday_offset(np.datetime64(start_date.strftime('%Y-%m-%d')), proj_days - 1 * (proj_days > 0), holidays=holidays_schedule, roll='forward')

    # Printing the result
    print(f'Developers need {proj_days} working day(s) to finish the job.')
    print(f'The estimated end date of the project is: {end_date}.')

In [2]:
caption = widgets.Label(value='Welcome to Project End Planner! Please choose the values to get the result.')
# Creating widgets
start_date_picker = widgets.DatePicker(
    description='Start date',
    value=datetime.datetime.now(),
    disabled=False
)

fe_devs_input = widgets.BoundedIntText(
    min=0,
    max=1000000,
    value=1,
    width=2000,
    description='FE devs:',
    disabled=False
)

be_devs_input = widgets.BoundedIntText(
    min=0,
    max=1000000,
    value=1,
    description='BE devs:',
    disabled=False
)

fe_task_input = widgets.BoundedIntText(
    min=0,
    max=1000000,
    value=20,
    description='FE task:',
    disabled=False
)

be_task_input = widgets.BoundedIntText(
    min=0,
    max=1000000,
    value=20,
    description='BE task:',
    disabled=False
)

country_picker = widgets.Dropdown(
    options=COUNTRIES,
    value='United States',
    description='Country:',
    disabled=False,
)

province_picker = widgets.Dropdown(
    options=(['-NONE-'] + PROVINCES['UnitedStates']),
    value='-NONE-',
    description='Province:',
    disabled=False,
)

observe_checkbox = widgets.Checkbox(
    value=True,
    description='Toggle observation',
    disabled=False
)

# Creating a dependency between country and provinces
def check_provinces(change):
    country = country_picker.value.replace(' ', '')
    if country in PROVINCES.keys():
        province_picker.options = ['-NONE-'] + PROVINCES[country]
    else:
        province_picker.options = ['-NONE-']
        province_picker.value = '-NONE-'

country_picker.observe(check_provinces)

# Creating HBox objects for correct displaying
ui_1 = widgets.HBox([start_date_picker])
ui_2 = widgets.HBox([fe_devs_input, fe_task_input])
ui_3 = widgets.HBox([be_devs_input, be_task_input])
ui_4 = widgets.HBox([country_picker, province_picker])
ui_5 = widgets.HBox([observe_checkbox])

# Assigning widgets values to function arguments
out = widgets.interactive_output(main, {'fe_devs':fe_devs_input, 'fe_task':fe_task_input, 'be_devs':be_devs_input, 'be_task':be_task_input, 'start_date':start_date_picker, 'country':country_picker, 'observe':observe_checkbox, 'state_prov':province_picker})

# Displaying widgets
display(caption, ui_1, ui_2, ui_3, ui_4, ui_5, out)

Label(value='Welcome to Project End Planner! Please choose the values to get the result.')

Output()

### Some speed test

In [3]:
%timeit main(fe_task=1000000, country='Ukraine', observe=True)

Developers need 125000 working day(s) to finish the job.
The estimated end date of the project is: 2518-02-17.
Developers need 125000 working day(s) to finish the job.
The estimated end date of the project is: 2518-02-17.
Developers need 125000 working day(s) to finish the job.
The estimated end date of the project is: 2518-02-17.
Developers need 125000 working day(s) to finish the job.
The estimated end date of the project is: 2518-02-17.
Developers need 125000 working day(s) to finish the job.
The estimated end date of the project is: 2518-02-17.
Developers need 125000 working day(s) to finish the job.
The estimated end date of the project is: 2518-02-17.
Developers need 125000 working day(s) to finish the job.
The estimated end date of the project is: 2518-02-17.
Developers need 125000 working day(s) to finish the job.
The estimated end date of the project is: 2518-02-17.
250 ms ± 9.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### The previous algorithm had a much worse performance (image):

In [4]:
from IPython.display import Image
Image("images/5.png")